In [107]:
import pandas as pd

dataset_name = "CNN-DailyMail"
# Treatment
df_deepseek_1_5b = pd.read_csv(f"./{dataset_name}/Treatment/results_deepseek-ai_DeepSeek-R1-Distill-Qwen-1.5B_{dataset_name}.csv")

df_gpt_neo_125m = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-125m_{dataset_name}.csv")
df_gpt_neo_1_3b = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-1.3B_{dataset_name}.csv")
df_gpt_neo_2_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_EleutherAI_gpt-neo-2.7B_{dataset_name}.csv")

df_opt_125m = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-125m_{dataset_name}.csv")
df_opt_1_3b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-1.3b_{dataset_name}.csv")
# df_opt_350m = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-350m_{dataset_name}.csv")
df_opt_2_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-2.7b_{dataset_name}.csv")
df_opt_6_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_facebook_opt-6.7b_{dataset_name}.csv")

df_vicuna_7b = pd.read_csv(f"./{dataset_name}/Treatment/results_lmsys_vicuna-7b-v1.5_{dataset_name}.csv")

# Control
df_smol_1_7b = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-1.7B-Instruct_{dataset_name}.csv")
df_smol_135m = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-135M-Instruct_{dataset_name}.csv")
df_smol_360m = pd.read_csv(f"./{dataset_name}/Control/results_HuggingFaceTB_SmolLM2-360M-Instruct_{dataset_name}.csv")

df_phi_3_5 = pd.read_csv(f"./{dataset_name}/Control/results_microsoft_Phi-3.5-mini-instruct_{dataset_name}.csv")


In [108]:
dfs = {
    "df_deepseek_1_5b": df_deepseek_1_5b,
    "df_gpt_neo_1_3b": df_gpt_neo_1_3b,
    "df_gpt_neo_125m": df_gpt_neo_125m,
    "df_gpt_neo_2_7b": df_gpt_neo_2_7b,
    "df_opt_1_3b": df_opt_1_3b,
    "df_opt_125m": df_opt_125m,
    "df_opt_2_7b": df_opt_2_7b,
    # "df_opt_350m": df_opt_350m,
    "df_opt_6_7b": df_opt_6_7b,
    "df_vicuna_7b": df_vicuna_7b,
    "df_smol_1_7b": df_smol_1_7b,
    "df_smol_135m": df_smol_135m,
    "df_smol_360m": df_smol_360m,
    "df_phi_3_5": df_phi_3_5
}

In [109]:
# df_deepseek_1_5b.head()
len(df_gpt_neo_1_3b)

1379

In [110]:
from scipy.stats import ttest_ind

# Perform Welch’s t-test; return a dictionary mapping score names to significance (1 or 0)
def t_test(df_suspicious_name, df_control_name, score_names):
    df_suspicious, df_control = dfs[df_suspicious_name], dfs[df_control_name]
    score_sig = dict()
    for score_name in score_names:
        scores_suspicious, scores_control = df_suspicious[score_name], df_control[score_name]
        stat, p_value = ttest_ind(scores_suspicious, scores_control, equal_var=False)
        print(f"comparing {score_name} between {df_suspicious_name} and {df_control_name}")
        print(f"mean ({df_suspicious_name}) = {scores_suspicious.mean()}, mean ({df_control_name}) = {scores_control.mean()}")
        print(f"t-statistic: {stat:.3f}, p-value: {p_value:.5f}")
        if p_value <= 0.05:
            print("Significant difference between suspicious and control groups.")
            score_sig[score_name] = f"1, {round(scores_suspicious.mean(), 4)}, {round(scores_control.mean(), 4)}"
        else:
            print("No significant difference detected.")
            score_sig[score_name] = "0"
        print("----------------------------------------------------------------")
    return score_sig


In [111]:
score_names = df_deepseek_1_5b.drop(['Unnamed: 0', 'Model', 'Task_Prefix', 'Dataset_Name', 'Model_Responses', 'Gold_Labels'], axis=1).columns
score_names = list(score_names)

In [112]:
gpt = ["df_gpt_neo_125m", "df_gpt_neo_1_3b", "df_gpt_neo_2_7b"]
# opt = ["df_opt_125m", "df_opt_350m", "df_opt_1_3b", "df_opt_2_7b", "df_opt_6_7b"]
opt = ["df_opt_125m", "df_opt_1_3b", "df_opt_2_7b", "df_opt_6_7b"]
vicuna = ["df_vicuna_7b"]
smol = ["df_smol_135m", "df_smol_360m", "df_smol_1_7b"]
phi = ["df_phi_3_5"]
treatment = gpt + opt + vicuna
control = smol + phi

In [113]:
import itertools
# doing t-test for every possible combination 
all_pairs = list(itertools.product(treatment, control))

total = 0 # count total model pairs
model_pair_analysis = [] # store all model pairs with resulting score indications

for suspicious, control in all_pairs: 
    model_pair_analysis.append((
        suspicious,
        control,
        t_test(suspicious, control, score_names)
    ))
    total+= 1

comparing cos_sim_scores between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.36740446940713706, mean (df_smol_135m) = 0.5210366576848054
t-statistic: -22.973, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 261.881798404641, mean (df_smol_135m) = 264.2610587382161
t-statistic: -0.424, p-value: 0.67162
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.10811419954355954, mean (df_smol_135m) = 0.11976023322244651
t-statistic: -4.717, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing rouge1_recall between df_gpt_neo_125m and df_smol_135m
m

In [114]:
# from collections import defaultdict

# score_proportion = defaultdict(int) # store count of total model pairs where score S is significantly difference
# score_sig_match = defaultdict(list) # store pairs that contribute to the counts above^

# for sus, con, vals in model_pair_analysis:
#     for m in vals.keys(): 
#         score_proportion[m] += vals[m]
#         if vals[m]:
#             score_sig_match[m].append((sus, con))

In [115]:
# model size wise comparison
def model_size_wise_compare(model_list):
    df_model = pd.DataFrame()
    for i in range(len(model_list)):
        for j in range(i+1, len(model_list)):
            score_sig = t_test(model_list[i], model_list[j], score_names)
            print(score_sig)
            df_model = pd.concat([df_model, pd.DataFrame(score_sig, index=[f"{model_list[i]} vs {model_list[j]}"])])
            print()
    return df_model
    
# gpt-neo
df_gpt_neo = model_size_wise_compare(gpt)

comparing cos_sim_scores between df_gpt_neo_125m and df_gpt_neo_1_3b
mean (df_gpt_neo_125m) = 0.36740446940713706, mean (df_gpt_neo_1_3b) = 0.3611307735266157
t-statistic: 0.878, p-value: 0.38005
No significant difference detected.
----------------------------------------------------------------
comparing levenshtein_distance between df_gpt_neo_125m and df_gpt_neo_1_3b
mean (df_gpt_neo_125m) = 261.881798404641, mean (df_gpt_neo_1_3b) = 257.3952139231327
t-statistic: 0.927, p-value: 0.35384
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_gpt_neo_125m and df_gpt_neo_1_3b
mean (df_gpt_neo_125m) = 0.10811419954355954, mean (df_gpt_neo_1_3b) = 0.10586281933580105
t-statistic: 0.911, p-value: 0.36249
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_recall between df_gpt_neo_125m and df_gpt_neo_1_3b
mean (df_gpt_neo_125m) = 0.328994218

In [116]:
df_gpt_neo

,cos_sim_scores,levenshtein_distance,rouge1_precision,rouge1_recall,rouge1_f1,rouge2_precision,rouge2_recall,rouge2_f1,rougeL_precision,rougeL_recall,rougeL_f1,Min_10.0%_Prob,Min_20.0%_Prob,Min_30.0%_Prob,Min_40.0%_Prob,Min_50.0%_Prob
df_gpt_neo_125m vs df_gpt_neo_1_3b,0,0,0,0,0,0,0,0,0,0,0,"1, 10.8609, 10.1481","1, 9.4261, 8.6304","1, 8.4367, 7.5887","1, 7.6085, 6.742","1, 6.8797, 6.017"
df_gpt_neo_125m vs df_gpt_neo_2_7b,0,0,0,0,0,0,0,0,0,0,0,"1, 10.8609, 10.0165","1, 9.4261, 8.4883","1, 8.4367, 7.4307","1, 7.6085, 6.576","1, 6.8797, 5.8512"
df_gpt_neo_1_3b vs df_gpt_neo_2_7b,0,0,0,0,0,0,0,0,0,0,0,"1, 10.1481, 10.0165","1, 8.6304, 8.4883","1, 7.5887, 7.4307","1, 6.742, 6.576","1, 6.017, 5.8512"


In [117]:
import os
os.makedirs(f"t_test/{dataset_name}")
df_gpt_neo.to_csv(f"t_test/{dataset_name}/gpt_neo_.csv")

In [118]:
# model size wise comparison
# opt
df_opt = model_size_wise_compare(opt)
df_opt.to_csv(f"t_test/{dataset_name}/opt.csv")

comparing cos_sim_scores between df_opt_125m and df_opt_1_3b
mean (df_opt_125m) = 0.30217984149496235, mean (df_opt_1_3b) = 0.29290115589076215
t-statistic: 1.199, p-value: 0.23071
No significant difference detected.
----------------------------------------------------------------
comparing levenshtein_distance between df_opt_125m and df_opt_1_3b
mean (df_opt_125m) = 270.1022480058013, mean (df_opt_1_3b) = 265.6874546773024
t-statistic: 0.898, p-value: 0.36952
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_opt_125m and df_opt_1_3b
mean (df_opt_125m) = 0.08469830495184266, mean (df_opt_1_3b) = 0.08205063343147455
t-statistic: 1.025, p-value: 0.30524
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_recall between df_opt_125m and df_opt_1_3b
mean (df_opt_125m) = 0.2633053082543672, mean (df_opt_1_3b) = 0.25357391557741454
t-statis

In [119]:
df_opt

,cos_sim_scores,levenshtein_distance,rouge1_precision,rouge1_recall,rouge1_f1,rouge2_precision,rouge2_recall,rouge2_f1,rougeL_precision,rougeL_recall,rougeL_f1,Min_10.0%_Prob,Min_20.0%_Prob,Min_30.0%_Prob,Min_40.0%_Prob,Min_50.0%_Prob
df_opt_125m vs df_opt_1_3b,0,0,0,0,0,0,"1, 0.0542, 0.0462",0,0,0,0,"1, 10.675, 9.9732","1, 9.1528, 8.4591","1, 8.1265, 7.4052","1, 7.29, 6.5571","1, 6.5675, 5.8374"
df_opt_125m vs df_opt_2_7b,"1, 0.3022, 0.2426",0,"1, 0.0847, 0.0724","1, 0.2633, 0.2268","1, 0.1247, 0.1073","1, 0.0161, 0.0109","1, 0.0542, 0.0375","1, 0.0242, 0.0165","1, 0.0627, 0.054","1, 0.1949, 0.1687","1, 0.0923, 0.0799","1, 10.675, 9.8604","1, 9.1528, 8.3292","1, 8.1265, 7.263","1, 7.29, 6.4135","1, 6.5675, 5.6969"
df_opt_125m vs df_opt_6_7b,"1, 0.3022, 0.2846",0,"1, 0.0847, 0.0717","1, 0.2633, 0.2367","1, 0.1247, 0.1072",0,0,0,"1, 0.0627, 0.0528","1, 0.1949, 0.1747","1, 0.0923, 0.0788","1, 10.675, 9.8381","1, 9.1528, 8.2602","1, 8.1265, 7.1737","1, 7.29, 6.3121","1, 6.5675, 5.5915"
df_opt_1_3b vs df_opt_2_7b,"1, 0.2929, 0.2426",0,"1, 0.0821, 0.0724","1, 0.2536, 0.2268","1, 0.1211, 0.1073","1, 0.0142, 0.0109","1, 0.0462, 0.0375","1, 0.0212, 0.0165","1, 0.0603, 0.054","1, 0.1857, 0.1687","1, 0.0888, 0.0799","1, 9.9732, 9.8604","1, 8.4591, 8.3292","1, 7.4052, 7.263","1, 6.5571, 6.4135","1, 5.8374, 5.6969"
df_opt_1_3b vs df_opt_6_7b,0,0,"1, 0.0821, 0.0717","1, 0.2536, 0.2367","1, 0.1211, 0.1072",0,0,0,"1, 0.0603, 0.0528",0,"1, 0.0888, 0.0788","1, 9.9732, 9.8381","1, 8.4591, 8.2602","1, 7.4052, 7.1737","1, 6.5571, 6.3121","1, 5.8374, 5.5915"
df_opt_2_7b vs df_opt_6_7b,"1, 0.2426, 0.2846",0,0,0,0,"1, 0.0109, 0.0142","1, 0.0375, 0.05","1, 0.0165, 0.0215",0,0,0,0,0,"1, 7.263, 7.1737","1, 6.4135, 6.3121","1, 5.6969, 5.5915"


In [120]:
df_smol = model_size_wise_compare(smol)
df_smol.to_csv(f"t_test/{dataset_name}/smol.csv")

comparing cos_sim_scores between df_smol_135m and df_smol_360m
mean (df_smol_135m) = 0.5210366576848054, mean (df_smol_360m) = 0.5341902976007045
t-statistic: -2.117, p-value: 0.03435
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_smol_135m and df_smol_360m
mean (df_smol_135m) = 264.2610587382161, mean (df_smol_360m) = 264.9253081943437
t-statistic: -0.118, p-value: 0.90625
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_smol_135m and df_smol_360m
mean (df_smol_135m) = 0.11976023322244651, mean (df_smol_360m) = 0.11937327076913014
t-statistic: 0.162, p-value: 0.87100
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_recall between df_smol_135m and df_smol_360m
mean (df_smol_135m) = 0.45914431407435713, mean (d

In [121]:
df_smol

,cos_sim_scores,levenshtein_distance,rouge1_precision,rouge1_recall,rouge1_f1,rouge2_precision,rouge2_recall,rouge2_f1,rougeL_precision,rougeL_recall,rougeL_f1,Min_10.0%_Prob,Min_20.0%_Prob,Min_30.0%_Prob,Min_40.0%_Prob,Min_50.0%_Prob
df_smol_135m vs df_smol_360m,"1, 0.521, 0.5342",0,0,"1, 0.4591, 0.4734",0,0,0,0,0,"1, 0.3567, 0.37",0,"1, 10.995, 10.3514","1, 9.2888, 8.6909","1, 8.1876, 7.6072","1, 7.313, 6.7468","1, 6.5659, 6.0211"
df_smol_135m vs df_smol_1_7b,"1, 0.521, 0.5433",0,0,0,0,0,0,0,0,0,0,"1, 10.995, 9.8498","1, 9.2888, 8.2055","1, 8.1876, 7.1014","1, 7.313, 6.2423","1, 6.5659, 5.5306"
df_smol_360m vs df_smol_1_7b,0,0,0,0,0,0,0,0,0,0,0,"1, 10.3514, 9.8498","1, 8.6909, 8.2055","1, 7.6072, 7.1014","1, 6.7468, 6.2423","1, 6.0211, 5.5306"


In [122]:
sus_con_pairs = [("df_gpt_neo_125m", "df_smol_135m"), ("df_opt_125m", "df_smol_135m")]

In [123]:
# sus vs. con (similar model sizes) comparison
def sus_con_wise_compare(pair_list):
    print(pair_list)
    df_model = pd.DataFrame()
    for pair in pair_list:
        score_sig = t_test(pair[0], pair[1], score_names)
        print(score_sig)
        df_model = pd.concat([df_model, pd.DataFrame(score_sig, index=[f"{pair[0]} vs {pair[1]}"])])
        print()
    return df_model


In [124]:
df_sus_con_compare = sus_con_wise_compare(sus_con_pairs)

[('df_gpt_neo_125m', 'df_smol_135m'), ('df_opt_125m', 'df_smol_135m')]
comparing cos_sim_scores between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.36740446940713706, mean (df_smol_135m) = 0.5210366576848054
t-statistic: -22.973, p-value: 0.00000
Significant difference between suspicious and control groups.
----------------------------------------------------------------
comparing levenshtein_distance between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 261.881798404641, mean (df_smol_135m) = 264.2610587382161
t-statistic: -0.424, p-value: 0.67162
No significant difference detected.
----------------------------------------------------------------
comparing rouge1_precision between df_gpt_neo_125m and df_smol_135m
mean (df_gpt_neo_125m) = 0.10811419954355954, mean (df_smol_135m) = 0.11976023322244651
t-statistic: -4.717, p-value: 0.00000
Significant difference between suspicious and control groups.
------------------------------------------------------------

In [125]:
df_sus_con_compare.to_csv(f"t_test/{dataset_name}/sus_con_compare.csv")